In [2]:
!pip install transformers torch sentencepiece
!pip install speechrecognition
!pip install pyttsx3
!pip install pyaudio



In [2]:
import speech_recognition as sr
import pyttsx3
from transformers import MarianMTModel, MarianTokenizer
import torch

# Load English-to-Spanish model
en_to_es_model_name = "Helsinki-NLP/opus-mt-en-es"
es_to_en_model_name = "Helsinki-NLP/opus-mt-es-en"

en_to_es_tokenizer = MarianTokenizer.from_pretrained(en_to_es_model_name)
en_to_es_model = MarianMTModel.from_pretrained(en_to_es_model_name)

es_to_en_tokenizer = MarianTokenizer.from_pretrained(es_to_en_model_name)
es_to_en_model = MarianMTModel.from_pretrained(es_to_en_model_name)

# Initialize recognizer and TTS engine
recognizer = sr.Recognizer()
engine = pyttsx3.init()

def translate_text(text, tokenizer, model):
    inputs = tokenizer.prepare_seq2seq_batch([text], return_tensors="pt", padding=True)
    translated = model.generate(**inputs)
    return tokenizer.batch_decode(translated, skip_special_tokens=True)[0]

def speak(text, lang='en'):
    engine.setProperty('rate', 150)
    engine.say(text)
    engine.runAndWait()

def listen(language_code="en-US"):
    with sr.Microphone() as source:
        print(f"Listening ({language_code})...")
        audio = recognizer.listen(source)
    try:
        return recognizer.recognize_google(audio, language=language_code)
    except sr.UnknownValueError:
        print("Could not understand audio.")
        return None
    except sr.RequestError as e:
        print(f"Request Error: {e}")
        return None

def real_time_conversation():
    print("Real-time Voice Translator Started. Ctrl+C to stop.\n")
    try:
        while True:
            print("👤 Spanish Speaker, your turn:")
            spanish_text = listen(language_code="es-ES")
            if spanish_text:
                print(f"Spanish: {spanish_text}")
                translated_to_english = translate_text(spanish_text, es_to_en_tokenizer, es_to_en_model)
                print(f"Translated to English: {translated_to_english}")
                speak(translated_to_english, lang='en')

            print("👤 English Speaker, your turn:")
            english_text = listen(language_code="en-US")
            if english_text:
                print(f"English: {english_text}")
                translated_to_spanish = translate_text(english_text, en_to_es_tokenizer, en_to_es_model)
                print(f"Translated to Spanish: {translated_to_spanish}")
                speak(translated_to_spanish, lang='es')

    except KeyboardInterrupt:
        print("\nConversation Ended.")

# Run the conversation
real_time_conversation()


Real-time Voice Translator Started. Ctrl+C to stop.

👤 Spanish Speaker, your turn:
Listening (es-ES)...
Could not understand audio.
👤 English Speaker, your turn:
Listening (en-US)...
English: hi how are you
Translated to Spanish: Hola, ¿cómo estás?
👤 Spanish Speaker, your turn:
Listening (es-ES)...
Spanish: pan
Translated to English: bread
👤 English Speaker, your turn:
Listening (en-US)...
English: what is the weather today
Translated to Spanish: lo que es el clima hoy
👤 Spanish Speaker, your turn:
Listening (es-ES)...
Spanish: buenos días
Translated to English: Good morning.
👤 English Speaker, your turn:
Listening (en-US)...
English: my name is Shweta
Translated to Spanish: Me llamo Shweta.
👤 Spanish Speaker, your turn:
Listening (es-ES)...
Could not understand audio.
👤 English Speaker, your turn:
Listening (en-US)...
Could not understand audio.
👤 Spanish Speaker, your turn:
Listening (es-ES)...
Spanish: grises
Translated to English: grey
👤 English Speaker, your turn:
Listening (en-US